In [1]:
from typing import AsyncIterable, TypeVar, Any, Iterable, Awaitable
from pydantic import BaseModel
import pure_cv as vc
import haskellian.either as E
import haskellian.asyn as hka
import haskellian.iterables
from moveread.annotations.images import ImageMeta, Source
from moveread.annotations.games import GameMeta
from moveread.sdk import MovereadAPI
from moveread.boxes.annotations import RobustExtractionResult
from moveread.bulk_extraction.scripts import extract, transfer, ExtractMeta
from moveread.bulk_extraction.scripts.extract_ import ExtractTask, run_task
from moveread.core import Game, CoreAPI, Image, ImageID, ops
from moveread.export.games import export_boxes
import requests
import robust_extraction as re
import robust_extraction.templates as ts

In [2]:
input = MovereadAPI.at('/home/m4rs/.core/llobregat23/')
output = MovereadAPI.at('/home/m4rs/.core/scans-extr-llobregat23/')

In [3]:
def filter_sources(game: Game, sources: set[Source]) -> Iterable[tuple[ImageID, Image]]:
  for id, image in game.images:
    meta = E.validate(image.meta, ImageMeta)
    if meta.match(lambda _: False, lambda m: m.source in sources):
      yield id, image

async def sources_of(games: AsyncIterable[Game], sources: set[Source]) -> AsyncIterable[tuple[ImageID, Image]]:
  async for game in games:
    for x in filter_sources(game, sources):
      yield x

a_games = [game for res in (await input.games.list()) if (game := res.unsafe()).startswith('a')]

async def prepare_all(games: list[str], input: MovereadAPI, output: MovereadAPI):
  for gameId in games:
    try:
      game = (await input.games.read(gameId)).unsafe()
      (await output.core.games.insert(gameId, game)).unsafe()
      yield game
    except E.IsLeft as e:
      print(f'Error with game IO: {gameId}:', e)

async def scan_tasks(games: list[str], input: MovereadAPI, output: MovereadAPI) -> AsyncIterable[ExtractTask]:
  async for imgId, image in sources_of(prepare_all(games, input, output), {'corrected-scan'}):
    img = await input.core.blobs.read(image.url)
    if E.is_right(img):
      yield ExtractTask(img.value, 'llobregat23', imgId)

async for i, task in hka.enumerate(scan_tasks(a_games, input, output)):
  print(f'\r{i} [{task.imageId}]', end='', flush=True)
  res = await run_task(task, output)
  if res.tag == 'left':
    print('Error:', res.value)

617 [ImageID(gameId='a/9/9', player=1, page=0, version=1)]]